In [3]:
import pandas as pd

In [4]:
import re

In [5]:
from gensim import models

C:\Users\HP\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [6]:
from gensim.models.doc2vec import TaggedDocument

In [7]:
import numpy as np
import scipy
import math
import random

import sklearn
from nltk.corpus import stopwords
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds

In [8]:
from nltk import sent_tokenize

In [9]:
articles_df = pd.read_csv('shared_articles.csv')

In [36]:
interactions_df = pd.read_csv('users_interactions.csv')

In [46]:
df_acrticles = articles_df[['contentId','title','text']]
df_inter = interactions_df[['contentId','personId']]

In [47]:
merged = pd.merge(df_inter, df_acrticles,how='inner', on=['contentId'])

In [48]:
texts = (merged).drop_duplicates('contentId')['text'].tolist() ; len(texts)

2987

In [49]:
titles = (merged).drop_duplicates('contentId')['title'].tolist() ; len(titles)

2987

In [50]:
ident = (merged).drop_duplicates('contentId')['contentId'].tolist() ; len(ident)

2987

### Build the Doc2vec Input Data

In [21]:
# Build the input for one text
def text2data(text,title,tag_id):
    text = text + title
    txt = sent_tokenize(text)
    words = []
    tags = []
    for sent in txt:
        words.append(re.findall(r"[\w*|\d*]\S*[\w|\d]|[,!?;()\d*\w]", sent))
        tags.append(str(tag_id))
    return words,tags

In [51]:
docs = []
for i in range(len(texts)):
    wds , tagged = text2data(texts[i],titles[i],ident[i])
    for st in range(len(wds)):
        docs.append(TaggedDocument(words=wds[st], tags=[tagged[st]]))

## Build the Doc2vec Model

In [56]:
model = models.Doc2Vec(vector_size=1000, window=4, min_count=5, workers=7, epochs=10,verbose = 1)

In [57]:
model.build_vocab(docs)

In [58]:
print ('Start training process...')
model.train(docs, total_examples=model.corpus_count, epochs=model.iter)

Start training process...


C:\Users\HP\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  from ipykernel import kernelapp as app


In [59]:
model.save("doc2vec_article_1000")

## Get Most similar Documents 

In [33]:
print(model.docvecs.most_similar([str(ident[5])],topn = 50))

[('-3759596852696130492', 0.16361592710018158), ('6044362651232258738', 0.15363657474517822), ('-7814257818245609902', 0.1331198513507843), ('-447851796385928420', 0.13288384675979614), ('-4333056764244640444', 0.13283848762512207), ('-4095842175692097942', 0.13279742002487183), ('4102576381061107965', 0.13213461637496948), ('-532999578436827210', 0.1304732710123062), ('-3077386995956755130', 0.13018062710762024), ('7000840123067725009', 0.12977038323879242), ('3989649621607163677', 0.12709620594978333), ('240943776326872028', 0.12684029340744019), ('-7518373517401484139', 0.1206541359424591), ('-2778760500673113802', 0.12061255425214767), ('310515487419366995', 0.12029175460338593), ('-6752460077509522132', 0.11998197436332703), ('-8189315446782327370', 0.11639401316642761), ('-8047634385511625486', 0.11588315665721893), ('-498521148402713730', 0.11571351438760757), ('-21036008762564671', 0.11361254751682281), ('-9099155556042679205', 0.11346578598022461), ('589254894302309232', 0.112